In [ ]:
'''
CAUTION:
All the clean event files in directory failed to clean!
always take this when doing evselect!
gtifile1 = 'EMOS1_gti_loband_2023-05-05.fits'
gtifile2 = 'EMOS1_gti_hiband_2023-05-05.fits'

'''

'''
This spectrum extraction pipeline is adapted by sas pipeline
https://www.cosmos.esa.int/web/xmm-newton/sas-thread-esasspec

'''
import numpy as np
import os
from glob import glob


# Set SAS environment
define workdir

In [5]:
gtifile1 = 'EMOS1_gti_loband_2023-05-05.fits'
gtifile2 = 'EMOS1_gti_hiband_2023-05-05.fits'
workdir  = '/stage/headat/yanling/xmm/reduction_230227/spectras/filtered_onlyhiband/101'
os.makedirs(workdir, exist_ok=True)
datadir = '/stage/headat/yanling/xmm/reduction_230227/101'
os.chdir(workdir)
# input_evtfile = f'{instrument}_hiband_loband_clean_{today}_fov.fits'


In [11]:
obs = 101
string = f'''
sasversion
export SAS_CCF=/stage/headat/yanling/xmm/0502020{obs}/ccf.cif
export SAS_ODF={glob(f'/stage/headat/yanling/xmm/0502020{obs}/*0502020{obs}_*SUM.SAS')[0]}
sasversion
sasversion
'''
print(string)



sasversion
export SAS_CCF=/stage/headat/yanling/xmm/0502020101/ccf.cif
export SAS_ODF=/stage/headat/yanling/xmm/0502020101/1378_0502020101_SCX00000SUM.SAS
sasversion
sasversion



# generate vignetted corrected, bkg subtracted images

In [ ]:
# get emosaic images (do it locally when extracting sb profile, and upload to sron server)
# also define spectra extraction region locally and upload to server
# regions should be in detector coordinates

### regions for shocks in A222
# shock*_phy_*01.reg

### regions for A222 radial profile


### regions for A223 radial profile




# extract QPB spectra

## 1. Use fwc data for both 101 and 201
LIVETIME before & after flare filtered:
EMOS1: 106.8 ks (101.2 ks)
EMOS2: 106.9 ks (102.1 ks)
EPN and OOT: 105 ks (102.3 ks)

evqpb (don't use, sron cif file don't contain PN EFF information, and the extracted fwc= exptime instead of 2 times of exp time!)


In [ ]:
## check the scaling of fwc
'''oofov area 
bkg
/stage/headat/yanling/xmm/fwc_dat_2023-05-10/events/E*_obs*01_bkg_clean_2023-05-10.fits
inst, cts, livetime, cts/livetime
mos1_closed_FF_2022_v2 125161 1886442.17854285 0.06634764713365265
mos2_closed_FF_2022_v2 136006 1859701.47908783 0.07313324290450635
pn_closed_EFF_2022_v2 39345 739987.328768998 0.053169829361067796
pn_closed_EFF_oot_2022_v2 35303 740268.032837234 0.04768948331416362
101 obs 
/stage/headat/yanling/xmm/reduction_230227/*01/{inst}_hiband_loband_clean_fov.fits
inst, cts, livetime, cts/livetime
EMOS1 3360 39004.5795996189 0.08614373067189345
EMOS2 4230 46478.1277415752 0.09101055067276771
EPN 1339 15959.1501955055 0.08390171052949275
Oot 1116 14977.9862565659 0.07450934864563513
201 obs 
inst, cts, livetime, cts/livetime
EMOS1 3360 39004.5795996189 0.0850
EMOS2 4230 46478.1277415752 0.0984
EPN 1339 15959.1501955055 0.0830
Oot 1116 14977.9862565659 0.0789
'''
qpb_oofov_ctr_backscales = {'101':[1.298, 1.244, 1.578, 1.562], '201':[1.281, 1.345, 1.561, 1.654]}

In [ ]:
# extract the spec at bkg regions
obs = 101
bkgdir = '/stage/headat/yanling/xmm/fwc_dat_2023-05-10/events'
bkgfiles = glob(f'{bkgdir}/E*_obs{obs}_bkg_clean_2023-05-10.fits')
print(bkgfiles)
f = open(f'qpb_bkg_{obs}.sh','w')
for bkgfile in bkgfiles:
    inst = bkgfile.split('/')[-1].split('_')[0]
    gtifile = f'{bkgdir}/../{inst}_bkg_gti_2023-05-10.fits'
    if 'MOS' in inst:
        regname = glob(f'../../regions/bkg_{obs}_all.reg')[0]
    else:
        regname = glob(f'../../regions/bkg_{obs}_pn.reg')[0]
    f1 = open(regname)
    lines = f1.readlines()
    f1.close()

    if 'MOS1' in bkgfile:
        express = f"(FLAG==0) && (PATTERN<=12) && CCDNR.ne.6"
    elif 'MOS2' in bkgfile:
        express = f"(FLAG==0) && (PATTERN<=12)"
    else: 
        express = f"(FLAG==0) && (PATTERN<=4)"
    for i, line in enumerate(lines):
        express = express + line[:-1]
    # generate the events of [corner(_oofov)] and [all fov without corner region(_fov)]
    if 'MOS' in bkgfile:
        f.write(f'''
evselect table={bkgfile} withspectrumset=Y spectrumset=qpb_{inst}_{obs}_bkg_spec_bin5.fits energycolumn=PI spectralbinsize=5 withspecranges=yes specchannelmin=0 specchannelmax=11999 expression="{express}&&gti({gtifile},TIME)"\n
        ''')
    else:
        f.write(f'''
evselect table={bkgfile} withspectrumset=Y spectrumset=qpb_{inst}_{obs}_bkg_spec.fits energycolumn=PI spectralbinsize=5 withspecranges=yes specchannelmin=0 specchannelmax=20479 expression="{express}&&gti({gtifile},TIME)"\n
        ''')
    # generate the corresponding images of previously generated events
    f.write(f'''
evselect table={bkgfile} xcolumn=X ycolumn=Y imagebinning=binSize ximagebinsize=80 yimagebinsize=80 withimageset=Y imageset=qbp_{inst}_bkg_{obs}_img.fits expression="{express}&&gti({gtifile},TIME)"\n''')

f.close()
# f = open('qpb_bkg.sh')
# print(f.readlines())
# f.close()

In [ ]:
# extract the spec at annuli
obs = 101
bkgdir = '/stage/headat/yanling/xmm/fwc_dat_2023-05-10/events'
bkgfiles = glob(f'{bkgdir}/E*_obs{obs}_bkg_clean_2023-05-10.fits')
f = open(f'qpb_spec_{obs}.sh','w')
for bkgfile in bkgfiles:
    for i in range(5):
        inst = bkgfile.split('/')[-1].split('_')[0]
        gtifile = f'{bkgdir}/../{inst}_bkg_gti_2023-05-10.fits'
        regname = glob(f'../../regions/shocks_phy_{obs}_reg{int(i+1)}*.txt')[0]
        f1 = open(regname)
        lines = f1.readlines()
        f1.close()
        if 'MOS1' in bkgfile:
            express = f"(FLAG==0) && (PATTERN<=12) && CCDNR.ne.6"
        elif 'MOS2' in bkgfile:
            express = f"(FLAG==0) && (PATTERN<=12)"
        else: 
            express = f"(FLAG==0) && (PATTERN<=4)"
        for j, line in enumerate(lines):    
            if len(lines)>3:
                if j<=1:
                    express = express + line[:-2]
                elif (j>1) and j<len(lines)-1:
                    express = express + "&&" + line[:-2]
                else:
                    express = express+ "&&" + line[:-1]
            else:
                if j<1:
                    express = express + line[:-2]
                else:
                    express = express + line[:-1]
    
        # generate the events of [corner(_oofov)] and [all fov without corner region(_fov)]
        if 'MOS' in bkgfile:
            f.write(f'''
evselect table={bkgfile} withspectrumset=Y spectrumset=qpb_{inst}_{obs}_reg{int(i+1)}_spec_bin5.fits energycolumn=PI spectralbinsize=5 withspecranges=yes specchannelmin=0 specchannelmax=11999 expression="{express}&&gti({gtifile},TIME)"\n
            ''')
        else:
            f.write(f'''
evselect table={bkgfile} withspectrumset=Y spectrumset=qpb_{inst}_{obs}_reg{int(i+1)}_spec.fits energycolumn=PI spectralbinsize=5 withspecranges=yes specchannelmin=0 specchannelmax=20479 expression="{express}&&gti({gtifile},TIME)"\n
            ''')
        # generate the corresponding images of previously generated events
        f.write(f'''
evselect table={bkgfile} xcolumn=X ycolumn=Y imagebinning=binSize ximagebinsize=80 yimagebinsize=80 withimageset=Y imageset=qpb_{inst}_{obs}_reg{int(i+1)}_img.fits expression="{express}&&gti({gtifile},TIME)"\n''')
f.close()
# f = open(f'qpb_spec_{obs}.sh')
# print(f.readlines())
# f.close()

calculate QPB spectras scales

scale QPB spectra 


In [ ]:
qpb_oofov_ctr_scales = {'101':[1.298, 1.244, 1.578, 1.562], '201':[1.281, 1.345, 1.561, 1.654]}
obs = '101'
f = open(f'scale_qpb_{obs}.sh', 'w')

for i, inst in enumerate(['EMOS1', 'EMOS2', 'EPN', 'Oot']):
    speclist = glob(f'qpb_{inst}_{obs}_*_spec.fits')
    for spec in speclist:
        f.write(f"mathpha expr='{spec} * {qpb_oofov_ctr_scales[obs][i]}' units=C outfil={spec.split('.')[0]}_scaled.fits exposure={spec} areascal=NULL clobber=yes ncomments=0\n")
f.close()

check scaled values

In [ ]:
from astropy.io import fits
with fits.open('qpb_EMOS1_201_reg3_spec_scaled.fits') as f:
    print(np.max(f[1].data['COUNTS']))
with fits.open('qpb_EMOS1_201_reg3_spec.fits') as f:
    print(np.max(f[1].data['COUNTS']))

In [ ]:
bkgdir = '/stage/headat/yanling/xmm/fwc_dat_2023-05-10/events'
datafile = f'{bkgdir}/EPN_bkg_clean_2023-05-10.fits'
express = '#XMMEA_16&&(PATTERN<=4)'
obs = '101'
inst = 'EPN'
print(f'evselect table={datafile} withspectrumset=Y spectrumset=EPN_101_oofov_qpb_spec.fits energycolumn=PI spectralbinsize=5 withspecranges=yes specchannelmin=0 specchannelmax=20479 expression="{express}"\n')

datadir = '/stage/headat/yanling/xmm/reduction_230227/101'
datafile = f'{datadir}/EPN_hiband_loband_clean_2023-05-05_oofov.fits'
express = '(PATTERN<=4)'
obs = '101'
inst = 'EPN'
print(f'evselect table={datafile} withspectrumset=Y spectrumset=EPN_101_oofov_data_spec.fits energycolumn=PI spectralbinsize=5 withspecranges=yes specchannelmin=0 specchannelmax=20479 expression="{express}"\n')

check spec extract regions

In [ ]:
from astropy.io import fits
from astropy.wcs import WCS
import matplotlib.pyplot as plt
from astropy.convolution import convolve
from astropy.convolution import Gaussian2DKernel
kernel = Gaussian2DKernel(x_stddev=1)
os.chdir(workdir)
os.makedirs('png', exist_ok=True)
for file in glob(f'*img.fits'):
    print(file)
    title = file
    hdu = fits.open(f'{file}')[0]
    image_data = hdu.data
    print(image_data.shape)
    fig = plt.figure(figsize = (6,6))
    ax = fig.add_axes([0.1, 0.1, 0.8, 0.8], projection=WCS(f'{file}'))
    ra = ax.coords[0]
    dec = ax.coords[1]

    ra.set_axislabel('RA')
    dec.set_axislabel('Dec')
    image_data = np.where(np.isfinite(np.log10(image_data)), image_data, np.nan)
    astropy_conv = convolve(image_data, kernel)
    ax.set_title(title)
    print(np.nanmax(np.log10(image_data)), np.nanmin(np.log10(image_data)[image_data>0]))
    ax.matshow(np.log10(astropy_conv),  cmap='gnuplot2',origin='lower')
    # plt.colorbar()
    plt.savefig(f'png/{os.path.basename(file).split(".")[0]}.png')
    # plt.show()
    plt.close()


# extract spectrums

In [14]:
# extract the spec at data regions
obs = 101
datadir = f'/stage/headat/yanling/xmm/reduction_230227/{obs}'
datafiles = glob(f'{datadir}/E*_hiband_loband_clean_fov.fits')
f = open(f'source_{obs}_bkg_spec.sh','w')
for datafile in datafiles:
    inst = datafile.split('/')[-1].split('_')[0]
    gtifile1 = f'{datadir}/{inst}_gti_loband_2023-05-05.fits'
    gtifile2 = f'{datadir}/EMOS2_gti_hiband_2023-05-05.fits'
    if 'MOS' in inst:
        regname = glob(f'../../regions/bkg_{obs}_all.reg')[0]
    else:
        regname = glob(f'../../regions/bkg_{obs}_pn.reg')[0]
    f1 = open(regname)
    lines = f1.readlines()
    f1.close()

    if 'MOS1' in datafile:
        express = f"(FLAG==0) && (PATTERN<=12) && CCDNR.ne.6"
    elif 'MOS2' in datafile:
        express = f"(FLAG==0) && (PATTERN<=12)"
    else: 
        express = f"(FLAG==0) && (PATTERN<=4)"
    for i, line in enumerate(lines):
        express = express + line[:-1]
    # generate the events of [corner(_oofov)] and [all fov without corner region(_fov)]
    if 'MOS' in datafile:
        f.write(f'''
evselect table={datafile} withspectrumset=Y spectrumset=source_{inst}_{obs}_bkg_spec_bin5.fits energycolumn=PI spectralbinsize=5 withspecranges=yes specchannelmin=0 specchannelmax=11999 expression="{express}&&gti({gtifile2},TIME)"\n
        ''')
    else:
        f.write(f'''
evselect table={datafile} withspectrumset=Y spectrumset=source_{inst}_{obs}_bkg_spec.fits energycolumn=PI spectralbinsize=5 withspecranges=yes specchannelmin=0 specchannelmax=20479 expression="{express}&&gti({gtifile2},TIME)"\n
        ''')
    # generate the corresponding images of previously generated events
    f.write(f'''
evselect table={datafile} xcolumn=X ycolumn=Y imagebinning=binSize ximagebinsize=80 yimagebinsize=80 withimageset=Y imageset=source_{inst}_bkg_{obs}_img.fits expression="{express}&&gti({gtifile2},TIME)"\n
    ''')

f.close()
f = open(f'source_{obs}_bkg_spec.sh')
print(f.readlines())
f.close()

['\n', 'evselect table=/stage/headat/yanling/xmm/reduction_230227/101/EMOS1_hiband_loband_clean_fov.fits withspectrumset=Y spectrumset=source_EMOS1_101_bkg_spec_bin5.fits energycolumn=PI spectralbinsize=5 withspecranges=yes specchannelmin=0 specchannelmax=11999 expression="(FLAG==0) && (PATTERN<=12) && CCDNR.ne.6&&(((X,Y) IN circle(14280.503,22120.496,3600))||((X,Y) IN ellipse(34143.931,35808.983,3022.7,5858.74,20.000129)))&&!((X,Y) IN ellipse(30976.894,40421.167,1177.58,732.6,148.48054))&&!((X,Y) IN ellipse(33292.262,40795.094,651.2,586.32,140.88808))&&!((X,Y) IN ellipse(32517.501,36298.293,267.96,238.84,67.291356))&&!((X,Y) IN ellipse(35119.686,39799.764,233.4,156.8,87.631986))&&!((X,Y) IN ellipse(35466.543,34487.918,566.26,498.58,115.08428))&&!((X,Y) IN ellipse(36753.658,34831.119,697.4,441.2,93.351365))&&!((X,Y) IN ellipse(34511.694,31562.732,444.88,357.78,101.90737))&&!((X,Y) IN ellipse(12634.451,24561.072,406.44,336.44,143.41351))&&!((X,Y) IN ellipse(14461.988,23707.064,296.82,20

In [ ]:
# extract the spec at annuli
obs = 101
datadir = f'/stage/headat/yanling/xmm/reduction_230227/{obs}'
datafiles = glob(f'{datadir}/E*_hiband_loband_clean_fov.fits')
f = open(f'source_{obs}_data_spec.sh','w')
for datafile in datafiles:
    for i in range(5):
        inst = datafile.split('/')[-1].split('_')[0]
        gtifile1 = f'{datadir}/{inst}_gti_loband_2023-05-05.fits'
        gtifile2 = f'{datadir}/EMOS2_gti_hiband_2023-05-05.fits'
        regname = glob(f'../../regions/shocks_phy_{obs}_reg{int(i+1)}*.txt')[0]
        f1 = open(regname)
        lines = f1.readlines()
        f1.close()
        if 'MOS1' in datafile:
            express = f"(FLAG==0) && (PATTERN<=12) && CCDNR.ne.6"
        elif 'MOS2' in datafile:
            express = f"(FLAG==0) && (PATTERN<=12)"
        else: 
            express = f"(FLAG==0) && (PATTERN<=4)"
        for j, line in enumerate(lines):    
            if len(lines)>3:
                if j<=1:
                    express = express + line[:-2]
                elif (j>1) and j<len(lines)-1:
                    express = express + "&&" + line[:-2]
                else:
                    express = express+ "&&" + line[:-1]
            else:
                if j<1:
                    express = express + line[:-2]
                else:
                    express = express + line[:-1]
    
        # generate the events of [corner(_oofov)] and [all fov without corner region(_fov)]
        if 'MOS' in datafile:
            f.write(f'''
evselect table={datafile} withspectrumset=Y spectrumset=source_{inst}_{obs}_reg{int(i+1)}_spec_bin5.fits energycolumn=PI spectralbinsize=5 withspecranges=yes specchannelmin=0 specchannelmax=11999 expression="{express}&&gti({gtifile2},TIME)"\n
            ''')
        else:
            f.write(f'''
evselect table={datafile} withspectrumset=Y spectrumset=source_{inst}_{obs}_reg{int(i+1)}_spec.fits energycolumn=PI spectralbinsize=5 withspecranges=yes specchannelmin=0 specchannelmax=20479 expression="{express}&&gti({gtifile2},TIME)"\n
            ''')
        # generate the corresponding images of previously generated events
        f.write(f'''
evselect table={datafile} xcolumn=X ycolumn=Y imagebinning=binSize ximagebinsize=80 yimagebinsize=80 withimageset=Y imageset=source_{inst}_{obs}_reg{int(i+1)}_img.fits expression="{express}&&gti({gtifile2},TIME)"\n''')
f.close()
# f = open('source_spec.sh')
# print(f.readlines())
# f.close()

extract source spectrums from oofov

In [6]:
# extract the spec at data regions
from glob import glob
obs = 101
datadir = f'/stage/headat/yanling/xmm/reduction_230227/{obs}'
datafiles = glob(f'{datadir}/E*_hiband_loband_clean_fov.fits')
f = open(f'source_{obs}_oofov_spec.sh','w')
for datafile in datafiles:
    inst = datafile.split('/')[-1].split('_')[0]
    gtifile1 = f'{datadir}/{inst}_gti_loband_2023-05-05.fits'
    gtifile2 = f'{datadir}/EMOS2_gti_hiband_2023-05-05.fits'


    if 'MOS1' in datafile:
        express = f"(PATTERN<=12) && CCDNR.ne.6"
    elif 'MOS2' in datafile:
        express = f"(PATTERN<=12)"
    else: 
        express = f"(PATTERN<=4)"

    # generate the events of [corner(_oofov)] and [all fov without corner region(_fov)]
    if 'MOS' in datafile:
        f.write(f'''
evselect table={datafile} withspectrumset=Y spectrumset=source_{inst}_{obs}_oofov_spec_bin5.fits energycolumn=PI spectralbinsize=5 withspecranges=yes specchannelmin=0 specchannelmax=11999 expression="#XMMEA_16&&{express}&&gti({gtifile2},TIME)"\n
        ''')
    else:
        f.write(f'''
evselect table={datafile} withspectrumset=Y spectrumset=source_{inst}_{obs}_oofov_spec.fits energycolumn=PI spectralbinsize=5 withspecranges=yes specchannelmin=0 specchannelmax=20479 expression="#XMMEA_16&&{express}&&gti({gtifile2},TIME)"\n
        ''')
    # generate the corresponding images of previously generated events
    f.write(f'''
evselect table={datafile} xcolumn=X ycolumn=Y imagebinning=binSize ximagebinsize=80 yimagebinsize=80 withimageset=Y imageset=source_{inst}_oofov_{obs}_img.fits expression="#XMMEA_16&&{express}&&gti({gtifile2},TIME)"\n
    ''')

f.close()
f = open(f'source_{obs}_oofov_spec.sh')
print(f.readlines())
f.close()

['\n', 'evselect table=/stage/headat/yanling/xmm/reduction_230227/101/EMOS1_hiband_loband_clean_fov.fits withspectrumset=Y spectrumset=source_EMOS1_101_oofov_spec_bin5.fits energycolumn=PI spectralbinsize=5 withspecranges=yes specchannelmin=0 specchannelmax=11999 expression="#XMMEA_16&&(PATTERN<=12) && CCDNR.ne.6&&gti(/stage/headat/yanling/xmm/reduction_230227/101/EMOS2_gti_hiband_2023-05-05.fits,TIME)"\n', '\n', '        \n', 'evselect table=/stage/headat/yanling/xmm/reduction_230227/101/EMOS1_hiband_loband_clean_fov.fits xcolumn=X ycolumn=Y imagebinning=binSize ximagebinsize=80 yimagebinsize=80 withimageset=Y imageset=source_EMOS1_oofov_101_img.fits expression="#XMMEA_16&&(PATTERN<=12) && CCDNR.ne.6&&gti(/stage/headat/yanling/xmm/reduction_230227/101/EMOS2_gti_hiband_2023-05-05.fits,TIME)"\n', '\n', '    \n', 'evselect table=/stage/headat/yanling/xmm/reduction_230227/101/EMOS2_hiband_loband_clean_fov.fits withspectrumset=Y spectrumset=source_EMOS2_101_oofov_spec_bin5.fits energycolum

# extract rmf and arf

extract rmf and arfs

In [10]:
obs = 101
f = open(f'gen_arf_rmf_{obs}.sh', 'w')
set3="withrmfset=true extendedsource=yes detmaptype=flat withbadpixcorr=Y modeleffarea=Y modelquantumeff=Y modelfiltertrans=Y"

for inst in ['EPN', 'EMOS1', 'EMOS2']:
    f.write(f'''
rmfgen spectrumset=source_{inst}_{obs}_oofov_spec_bin5.fits rmfset=source_{inst}_{obs}_bkg.rmf detmaptype=flat extendedsource=yes badpixlocation={inst}_hiband_loband_clean_fov.fits\n
        ''') # arfgen spectrumset=source_{inst}_{obs}_bkg_spec_bin5.fits {set3} rmfset=source_{inst}_{obs}_bkg.rmf arfset=source_{inst}_{obs}_bkg.arf badpixlocation={inst}_hiband_loband_clean_fov.fits\n
#     for reg in ['oofov']:#['reg1', 'reg2', 'reg3', 'reg4', 'reg5']:
#         f.write(f'''
# rmfgen spectrumset=source_{inst}_{obs}_{reg}_spec_bin.fits rmfset=source_{inst}_{obs}_{reg}.rmf detmaptype=flat extendedsource=yes badpixlocation={inst}_hiband_loband_clean_fov.fits\n
#         ''')# arfgen spectrumset=source_{inst}_{obs}_{reg}_spec.fits {set3} rmfset=source_{inst}_{obs}_{reg}.rmf arfset=source_{inst}_{obs}_{reg}.arf badpixlocation={inst}_hiband_loband_clean_fov.fits\n
f.close()
# f = open(f'gen_arf_rmf_{obs}.sh')
# print(f.readlines())

1. Specifying an external detector map for an extended source
skip it if the detmaptype=flat

In [ ]:
#to create a detector coordinate image covering the region  $-1500<DETX<+1500$ and  $-1500<DETY<+1500$ of size 20x20 pixels use the command:
obs = 101
datadir = f'/stage/headat/yanling/xmm/reduction_230227/{obs}'
datafiles = glob(f'{datadir}/E*_hiband_loband_clean_fov.fits')
f = open(f'det_img_{obs}.sh','w')
for datafile in datafiles:
    inst = datafile.split('/')[-1].split('_')[0]
    f.write(f'''
evselect table={datafile} destruct=false withfilteredset=true withimageset=true imageset=detmap_{inst}_{obs}_img.fits xcolumn=DETX ycolumn=DETY withxranges=true ximagemin=-1500 ximagemax=1500 withyranges=true yimagemin=-1500 yimagemax=1500 imagebinning='imageSize' ximagesize=400 yimagesize=400 writedss=true updateexposure=true''')
f.close()
f = open(f'det_img_{obs}.sh')
print(f.readlines())
f.close()

combine spectra
https://www.cosmos.esa.int/web/xmm-newton/sas-thread-epic-merging

In [ ]:
 epicspeccombine pha="src_spectrum_01.ds src_spectrum_02.ds src_spectrum_03.ds"\
   bkg="bkg_spectrum_01.ds bkg_spectrum_02.ds bkg_spectrum_03.ds" \
   rmf="response_01.rmf response_02.rmf response_03.rmf" \
   arf="arf_01.arf arf_02.arf arf_03.arf" \
   filepha="src_spectrum_grp.ds" \
   filebkg="bkg_spectrum_grp.ds" \
   filersp="response_grp.rmf" \
   allowHEdiff=yes